In [1]:
!pip install langchain
!pip install streamlit
!pip install openai
!pip install aleph_alpha_client
!pip install replicate
!pip install huggingface_hub
!pip install chromadb
!pip install tiktoken

You should consider upgrading via the '/Users/jonahusadi/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.
     |████████████████████████████████| 8.9 MB 15.5 MB/s eta 0:00:01
     |████████████████████████████████| 813 kB 6.7 MB/s eta 0:00:01     |███████████████████████████     | 686 kB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 238 kB 7.5 MB/s eta 0:00:01
     |████████████████████████████████| 162 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 22.6 MB 569 kB/s eta 0:00:01
     |████████████████████████████████| 164 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 184 kB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 10.8 MB 618 kB/s eta 0:00:01    |██████████████                  | 4.8 MB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.8 MB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 9.5

In [6]:
import os
# Setup API keys
os.environ["OPENAI_API_KEY"] = "x"


# Quick Intro to LangChain

What are
- Prompts
- Chains
- Indexes

Other important ideas I won't cover now:
- Memory
- Agents

In [8]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

template = """
I want you to act as a naming consultant for new companies.
What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

llm = OpenAI(temperature=0)

chain = LLMChain(llm=llm, prompt=prompt)

result = chain.run("colorful sneakers")

print(result)


Rainbow Kicks.


In [40]:
llm("What would be a good company name?")

'\n\nVistaTech Solutions.'

In [47]:
from demo.chains.oncall_agent.runbook_documents import DOCS

for doc in DOCS:
    print(doc)
    print("")

page_content='How to Handle a SegFault:\n\nWhen the service encounters a segmentation fault, the proper solution is to restart the service. \nThis can be done by providing the following command to the VM: \n\n```\nsudo service restart\n```\n' metadata={'error': 'segmentation fault'}

page_content='How to Handle a Timeout:\n\nWhen the service hits timeouts, we need to investigate whether or not there is enough space left on the device.\nTo do this, we need to run the following command:\n\n```\nsudo df\n```\n' metadata={'error': 'timeout'}

page_content='How to Handle a Disk Error:\n\nDisk errors mean the entire VM has reached a bad state. We need to reboot the entire VM by running this command: \n\n```\nsudo reboot\n```\n' metadata={'error': 'disk error'}



In [3]:
from project.chains.environment_agent.runbook_documents import DOCS
for doc in DOCS:
    print(doc)
    print("")

page_content="\nLet it grow.\nMore than 70 towns across the U.S. have signed on to relax maintenance rules for property owners during the month of May as part of a movement meant to feed local bee populations for the upcoming growing season.\nCalled “No Mow May,” the initiative started in the United Kingdom and was later adopted by Appleton, Wisconsin, which became the first U.S. town to implement it in 2020. This year, dozens of communities across the U.S. have adopted No Mow May to encourage citizens to let bee-friendly plants grow in their yards.\n“Pesky weeds like clovers and dandelions are like cheeseburgers for bees,” said Israel Del Toro, an assistant biology professor at Lawrence University and city council member in Appleton.\nAnd Appleton has seen the bee benefits, Del Toro said. Since the project began, No Mow May lawns in Appleton showed a fivefold increase in bee abundance and a threefold increase in bee diversity compared to nearby parkland that was mowed regularly.\nBees

In [9]:
from project.chains.environment_agent.oncall_agent_chain import OncallChain
from project.chains.environment_agent.runbook_documents import create_index

index = create_index()
topic = "bees"

docs = index.similarity_search(topic, k=1)
inputs = [{"article": doc.page_content, "topic": topic} for doc in docs]
chain = OncallChain.from_llm(llm)
result = chain.apply(inputs)

print(result)

[{'text': ' Summarize the article about bees and their impact on the environment.\n\nThe article discusses the importance of bees in the U.S., as they pollinate $15 billion worth of crops and help farmers produce about one-third of all food eaten by Americans. However, global bee populations are in danger due to factors such as habitat loss, overuse of chemical pesticides and herbicides, disease and climate change. To help bees, homeowners can reduce the risk of disturbing soil for bees that nest in the ground by not mowing their lawns, which also has the added benefit of reducing local air pollution.'}]


# Data Generation and Auto-Evaluation



Two important insights:

* Chains can be prompted to produce synthetic data
* Chains can evaluate output from other LLMs

See [Auto-evaluator](https://autoevaluator.langchain.com/) for an example of evaluating question answering on documents.

# Example 1: Sales Email Generation

In [21]:
from demo.synthesizers.json_synthesizer.synthesize_data import synthesize

text_input = """
The goal of this language model task is to generate cold sales emails.
The input is a JSON containing a few details about a customer.

An example of the input would be: 
```
{{
    "Name": "Artur Moczulski",
    "Title": "CTO",
    "Company": "Scout AI",
}}
```
Your prompts should follow this JSON format.
Everything between the ``` must be valid json.
Generate one such JSON input.
"""

synthesized_data = synthesize(text_input, 1)

In [22]:
print(synthesized_data[0]['inputs'])

[{'Name': 'John Smith', 'Title': 'CEO', 'Company': 'ABC Inc.'}]


In [23]:
from demo.chains.sdr_agent.generate_email import generate_email

email = generate_email(synthesized_data)

2023-05-27 13:34:18.813 No runtime found, using MemoryCacheStorageManager
2023-05-27 13:34:18.818 No runtime found, using MemoryCacheStorageManager


In [24]:
print(email[0])

Subject: AI-powered LLM evaluation for ABC Inc.

Dear John,

I hope this email finds you well. My name is [Your Name] and I am a Sales Development Representative at [Your Company], a startup that specializes in helping software development teams evaluate LLMs for use in their products.

I came across ABC Inc. and was impressed by your company's innovative approach to [mention something specific about their business]. As a CEO, I understand that you are always looking for ways to improve your products and stay ahead of the competition.

That's where our AI-powered LLM evaluation comes in. Our technology can help your development team evaluate LLMs quickly and accurately, saving you time and resources. Our platform is user-friendly and can be easily integrated into your existing workflow.

I would love to schedule a brief call with you to discuss how our technology can benefit ABC Inc. and answer any questions you may have. Please let me know if this is something you would be interested 

In [25]:
from demo.evaluators.cold_email_binary_evaluator.evaluate_email import evaluate

result = evaluate(synthesized_data[0]['inputs'], email)

2023-05-27 13:34:36.206 No runtime found, using MemoryCacheStorageManager
2023-05-27 13:34:36.209 No runtime found, using MemoryCacheStorageManager


In [26]:
print(result)

[{'text': 'SATISFACTORY. The SDR addressed the customer by name and title, and mentioned something specific about their business. The email is clear and convincing, highlighting the benefits of the technology and offering to schedule a call to discuss further.'}]


In [27]:
from langchain.llms import Replicate
DOLLY_MODEL = "replicate/dolly-v2-12b:ef0e1aefc61f8e096ebe4db6b2bacc297daf2ef6899f0f7e001ec445893500e5"

email2 = generate_email(synthesized_data, Replicate(model=DOLLY_MODEL))

ValidationError: 1 validation error for Replicate
__root__
  Did not find REPLICATE_API_TOKEN, please add an environment variable `REPLICATE_API_TOKEN` which contains it, or pass  `REPLICATE_API_TOKEN` as a named parameter. (type=value_error)

In [ ]:
print(email2[0])

In [ ]:
result = evaluate(synthesized_data[0]['inputs'], email2)

In [29]:
print(result[0]['text'])

SATISFACTORY. The SDR addressed the customer by name and title, and mentioned something specific about their business. The email is clear and convincing, highlighting the benefits of the technology and offering to schedule a call to discuss further.


In [30]:
from demo.evaluators.cold_email_comparator_evaluator.evaluate_email import evaluate

result = evaluate(synthesized_data[0]['inputs'], email, email2)

2023-05-27 13:34:53.402 No runtime found, using MemoryCacheStorageManager


NameError: name 'email2' is not defined

In [31]:
print(result[0]['text'])

SATISFACTORY. The SDR addressed the customer by name and title, and mentioned something specific about their business. The email is clear and convincing, highlighting the benefits of the technology and offering to schedule a call to discuss further.


# Example 2: Oncall Bot

In [32]:
from demo.chains.oncall_agent.runbook_documents import create_index
from demo.chains.oncall_agent.generate_command import generate_command

error = "Error: segfault"
command = generate_command(error, index)

In [33]:
print(command)

[{'text': ' sudo service restart'}]


In [34]:
from demo.evaluators.oncall_action_evaluator.evaluate_action import evalaute

result = evalaute(error, command, index)

[{'text': ' SUCCESS\n\nThe Site Reliability Engineer correctly followed the runbook and provided the correct command to restart the service. Therefore, they should be given a grade of SUCCESS.'}]


# Using HuggingFace